In [3]:
def Date_sep_Name(Date_Name):
    Date_Name = Date_Name.dropna(how='all').reset_index(drop=True)
    Date = Date_Name[0]
    Name = Date_Name[1]
    
    return Date, Name

In [6]:
def Date_Name(date_name):
    date_name = date_name.dropna().reset_index(drop=True)
    date_name = date_name.str.upper()
    tags = date_name.str.findall(r'[A-Z]{2,3}')
    dates = date_name.str.findall(r'\d{1,2}/\d{1,2}/\d{2,4}')
    
#     for tag in tags:
#         if tag[0] == 'FCA' or tag[0] == 'MIL' or tag[0] == 'CAI' or tag[0] =='ISO':
#             tags = tags.drop(tag)
#     print(tags)
    
    tags = pd.Series([x.pop(0) for x in tags])
    dates = pd.Series([x.pop(0) if x else np.NaN for x in dates ])
    dates = dates.fillna(method='ffill')
    dates = dates.fillna(method='bfill')

    
    
        
    return dates, tags

In [8]:
def Date_Name_Descripcion(series, rows_per_row):
    
    def separador(string):
        not_names = ['FCA', 'MIL', 'CAI', 'ISO', 'KLG', 'PMK', 'TRC', 'CA', 'ASA', 'PMQ', 'VI', 'CV']
        date = regex.findall(r'^\d{1,2}/\d{1,2}/\d{4}', string)

        tag = regex.findall(r'\b[A-Z]{2,3}', string)

        if date:
            date = date.pop(0)
            string = string.replace(date, '')
        else:
            date = np.NaN
        if tag:
            if tag[0] not in not_names:
                tag = tag.pop(0)
                string = string.replace(tag, '')
            else:
                tag = np.NaN
        else:
            tag = np.NaN

        string = string.lstrip()
        return date, tag, string
    
    
    aux2 = series.dropna().reset_index(drop=True)
    aux2 = aux2.apply(lambda x: separador(x))
    aux2 = aux2.apply(pd.Series)
    aux2.columns = ['Date', 'Name', 'Description']
    Date_Name = aux2[['Date', 'Name']]
    Date_Name = Date_Name.dropna(subset=['Name']).reset_index(drop=True)
    Date = Date_Name['Date']
    Date = Date.fillna(method='ffill')
    Date = Date.fillna(method='bfill')
    Name = Date_Name['Name']
    Descripcion = aux2['Description'].dropna().reset_index(drop=True)
    
    x = 0
    descripcion = []
    for row, count in enumerate(rows_per_row):
        descripcion.append([*Descripcion[x:count+x]])
        x += count
    
    descripcion = pd.Series(descripcion)
    descripcion = descripcion.str.join(' ')
    
    descripcion = descripcion.reset_index(drop=True)
    Date = Date.reset_index(drop=True)
    Name = Name.reset_index(drop=True)
    
    return Date, Name, descripcion

In [20]:
def Remove_Date_Name(series):
    def separador(string):

        date = regex.findall(r'^\d{1,2}/\d{1,2}/\d{4}', string)

        tag = regex.findall(r'\b[A-Z]{2,3}', string)

        if date:
                date = date.pop(0)
                string = string.replace(date, '')
        else:
            date = np.NaN
        if tag:
                tag = tag.pop(0)
                string = string.replace(tag, '')
        else:
            tag = np.NaN

        string = string.lstrip()
        return string
    
    aux2 = series.dropna().reset_index(drop=True)
    aux2 = aux2.apply(lambda x: separador(x))
    
    return aux2

In [13]:
def Descripcion_func(descripcion, rows_per_row):
    descripcion = descripcion.dropna().reset_index(drop=True)
    
    x = 0
    Descripcion = []
#     print('\nDescription:\n')

    for row, count in enumerate(rows_per_row):
#         print(*descripcion[x:count+x])
        Descripcion.append([*descripcion[x:count+x]])
        x += count
    Descripcion = pd.Series(Descripcion)
    Descripcion = Descripcion.str.join(' ')
    
    return Descripcion

In [25]:
def Descripcion_Hours_Rates(series, rows_per_row):
    series = series.reset_index(drop=True)
    
    aux = series.str.findall(r' \d{1,2}\.\d{1,2}$')
    aux = pd.Series(aux)
    aux = pd.Series([x.pop(0) if x else np.NaN for x in aux ])  
    aux = aux.apply(str)
    aux = aux.apply(lambda x: x.strip())
    aux = aux.replace('nan', np.NaN)
    hours = aux.apply(float).dropna()
    
    series = series.str.replace('NO CHARGE', '00.00')
    
    aux3 = series.str.findall(r'\d{3}.\d{2}/hr')
    aux3 = pd.Series([x.pop(0) if x else np.NaN for x in aux3 ]) 
    rates = aux3.dropna().str.replace('/hr', '')
    rates = rates.str.replace(' ', '.')
    rates = rates.apply(float)
    
    def separador(string, hour, rate):
        if pd.isna(hour):
            if not pd.isna(rate):
                string = string.replace(rate, '')
        else:
            
            string = string.replace(hour, '')
        return string
    
    dfs = pd.concat([series.rename('descripcion'), aux.rename('hours'), aux3.rename('rates')], axis=1)
    
    descripcion = dfs.apply(lambda x: separador(x['descripcion'], x['hours'], x['rates']), axis=1)
    filterx = [True if len(x) == 0 else False for x in descripcion]
    descripcion.drop(labels = descripcion[filterx].index, inplace=True)
    
    Descripcion = []
    x = 0
    for row, count in enumerate(rows_per_row):
#         print(*descripcion[x:count+x])
        Descripcion.append([*descripcion[x:count+x]])
        x += count
    Descripcion = pd.Series(Descripcion)
    Descripcion = Descripcion.str.join(' ')
    
    hours = hours.reset_index(drop=True)
    rates = rates.reset_index(drop=True)
    
    return Descripcion, hours, rates

In [7]:
def Descripcion_Hours_Rates_Totals(series, rows_per_row):
    series = series.reset_index(drop=True)
    
    aux = series.str.findall(r' \d{1,2}\.\d{2} ')
    aux = pd.Series(aux)
    aux = pd.Series([x.pop(0) if x else np.NaN for x in aux ])  
    aux = aux.apply(str)
    aux = aux.apply(lambda x: x.strip())
    aux = aux.replace('nan', np.NaN)
    hours = aux.apply(float).dropna()
    
    series = series.str.replace('NO CHARGE', '00.00')
    
    aux2 = series.str.findall(r'\d?,?\d{2,3}\.\d{2}$')
    aux2 = pd.Series([x.pop(0) if x else np.NaN for x in aux2 ]) 
    totals = aux2.dropna().reset_index(drop=True)
    totals = totals.apply(str)
    totals = totals.str.replace('^00.00', 'nan')

    totals = totals.str.replace(',', '')
    totals = totals.apply(float)
    
    aux3 = series.str.findall(r'\d{3}\.\d{2}.hr')
    aux3 = pd.Series([x.pop(0) if x else np.NaN for x in aux3 ]) 
    rates = aux3.dropna().str.replace('.hr', '')
    rates = rates.apply(float)


    def separador(string, hour, rate, total):
        if pd.isna(hour):
            if not pd.isna(rate):
                string = string.replace(rate, '')
        else:
            
            string = string.replace(hour + ' ' + total, '')
        return string   
            
    dfs = pd.concat([series.rename('descripcion'), aux.rename('hours'), aux3.rename('rates'),
                     aux2.rename('totals')], axis=1)
    
    descripcion = dfs.apply(lambda x: separador(x['descripcion'], x['hours'], x['rates'], x['totals']), axis=1)
    filterx = [True if len(x) == 0 else False for x in descripcion]
    descripcion.drop(labels = descripcion[filterx].index, inplace=True)
    
    descripcion = descripcion.reset_index(drop=True)
    hours = hours.reset_index(drop=True)
    rates = rates.reset_index(drop=True)
    totals = totals.reset_index(drop=True)


    Descripcion = []
    x = 0
#     print('\nDescription:\n')
    for row, count in enumerate(rows_per_row):
#         print(*descripcion[x:count+x])
        Descripcion.append([*descripcion[x:count+x]])
        x += count
    Descripcion = pd.Series(Descripcion)
    Descripcion = Descripcion.str.join(' ')    
#     print('\nHours:\n')
#     for hour in hours:
#         print(hour)
#     print('\nRates:\n')
#     for rate in rates:
#         print(rate)
#     print('\nTotals:\n')
#     for total in totals:
#         print(total)
        
    return Descripcion, hours, rates, totals

In [16]:
def Hours_Rates(Hours_Rate):
    Hours_Rate = Hours_Rate.dropna().reset_index(drop=True)
    rates = Hours_Rate.drop(range(0, len(Hours_Rate), 2),axis=0)
    hours = Hours_Rate.drop(range(1, len(Hours_Rate), 2),axis=0)
    
    hours = hours.str.replace(' ', '.')
    hours = hours.str.replace('\.\.', '.')


    rates = rates.str.replace('i', '/')
    rates = rates.str.replace('/[hn]r', '')
    rates = rates.str.replace(' ', '.')
    rates = rates.apply(float)
    
    hours = hours.reset_index(drop=True)
    rates = rates.reset_index(drop=True)
    
    return hours, rates

In [9]:
def Hours_Rates_Totals(Hours_Rate_Total):
    Hours_Rate_Total = Hours_Rate_Total.dropna().reset_index(drop=True)
    rates = Hours_Rate_Total.drop(range(0, len(Hours_Rate_Total), 2),axis=0)
    hours_total = Hours_Rate_Total.drop(range(1, len(Hours_Rate_Total), 2),axis=0)
    
    rates = rates.str.replace('[/i]hr', '')
    rates = rates.str.replace(' ', '.')
    rates = rates.apply(float)
    
    aux = hours_total.str.findall('\d{1,2}\.\d{1,2}')
    hours  = [x.pop(0) for x in aux]
    hours = pd.Series(hours)
    totals = [x.replace(hour, '', 1) for x, hour in zip(hours_total, hours)]
    totals = pd.Series(totals)
    totals = totals.str.replace(',', '')
    totals = totals.str.replace('NO CHARGE', 'nan')
    totals = totals.str.replace('NO CH\.ARGE', 'nan')
    
    totals = totals.reset_index(drop=True)
    rates = rates.reset_index(drop=True)
    hours = hours.reset_index(drop=True)
    
    totals = totals.apply(float)
        
    return hours, rates, totals

In [18]:
def Totals_func(Totals):
    Totals = Totals.dropna().reset_index(drop=True)
    Totals = Totals.apply(str)
    Totals = Totals.str.replace(',', '')
    Totals = Totals.str.replace('NO CHARGE', 'nan')
    Totals = Totals.str.strip()
    Totals = Totals.apply(float)
    
    Totals = Totals.reset_index(drop=True)
    return Totals

In [5]:
def dndhrt(df, rows_per_row):
    Date, Name, _ = Date_Name_Descripcion(df.loc[:, 0], rows_per_row)
    aux_series = Remove_Date_Name(df.loc[:, 0])
    Descripcion, Hours, Rates, Totals = Descripcion_Hours_Rates_Totals(aux_series, rows_per_row)
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [14]:
def dndhr_t(df, rows_per_row):
    Date, Name, _ = Date_Name_Descripcion(df.loc[:, 0], rows_per_row)
    aux_series = Remove_Date_Name(df.loc[:, 0])
    Descripcion, Hours, Rates = Descripcion_Hours_Rates(aux_series, rows_per_row)
    Totals = Totals_func(df.loc[:, 1])
#     print(Date, Name, Descripcion, Hours, Rates, Totals)
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [10]:
def dnd_hrt(df, rows_per_row):
    Date, Name, Descripcion = Date_Name_Descripcion(df.loc[:, 0], rows_per_row)
    Hours, Rates, Totals = Hours_Rates_Totals(df.loc[:, 1])
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [16]:
def dnd_hr_t(df, rows_per_row):
    Date, Name, Descripcion = Date_Name_Descripcion(df.loc[:, 0], rows_per_row)
    Hours, Rates = Hours_Rates(df.loc[:, 1])
    Totals = Totals_func(df.loc[:, 2])
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [12]:
def dn_dhrt(df, rows_per_row):
    Date, Name = Date_Name(df.loc[:, 0])
    Descripcion, Hours, Rates, Totals = Descripcion_Hours_Rates_Totals(df.loc[:, 1], rows_per_row)
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
#     print(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [11]:
def dn_d_hrt(df, rows_per_row):
    Date, Name = Date_Name(df.loc[:, 0])
    Descripcion = Descripcion_func(df.loc[:, 1], rows_per_row)
    Hours, Rates, Totals = Hours_Rates_Totals(df.loc[:, 2])
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [18]:
def dn_d_hr_t(df, rows_per_row):
    Date, Name = Date_Name(df.loc[:, 0])
    Descripcion = Descripcion_func(df.loc[:, 1], rows_per_row)
    Hours, Rates = Hours_Rates(df.loc[:, 2])
    Totals = Totals_func(df.loc[:, 3])
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [17]:
def d_n_d_hr_t(df, rows_per_row):
    date_name = df[[0, 1]].dropna(how='all').reset_index(drop=True)
    Date = date_name[0]
    Name = date_name[1]
    Descripcion = Descripcion_func(df.loc[:, 2], rows_per_row)
    Hours, Rates = Hours_Rates(df.loc[:, 3])
    Totals = Totals_func(df.loc[:, 4])
    df_load = LOAD(Date, Name, Descripcion, Hours, Rates, Totals)
    return df_load

In [9]:
def LOAD(Date, Name, Descripcion, Hours, Rates, Totals):
    df = pd.concat([Date.rename('Date'), Name.rename('Names'), Descripcion.rename('Description'),
                     Hours.rename('Hours'), Rates.rename('Rates'), Totals.rename('Totals')], axis=1)
    df = df.dropna().reset_index(drop=True)
    return df

In [ ]:
def Organizador(df, rows_numbs=None, split_numbs=None):
    for row_num, split_num in zip(rows_numbs, split_numbs):
        aux = df.loc[row_num, 0].split(' ', maxsplit = split_num)
        date = ''
        if split_num == 1:
            name = aux[0]
            string = aux[1]

        elif split_num == 2:
            date = aux[0]
            name = aux[1]
            string = aux[2]
        else:
            raise ValueError('Split no válido')

        if pd.isna(df.loc[row_num, 1]):
            df.loc[row_num, 1] = string
            df.loc[row_num, 0] = date + ' ' + name if date else name
        else:
            df.loc[row_num, 1] = string + ' ' + df.loc[row_num, 1]
            df.loc[row_num, 0] = date + ' ' + name if date else name